# Movie Rec Sys

## Collaborative Filtering

Importing required modules 

In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Acquiring the Data


In [ ]:
!wget -O moviedataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

loading the csv files

In [2]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings_sample.csv')

Removing year from movie names and add it to a seprate column  using regex \d\d\d\d => match 4 digits

In [3]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

/tmp/ipykernel_6426/1075324330.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Removing genres columns because its not needed in collaborative filtering

In [4]:
movies_df = movies_df.drop('genres', 1)
movies_df.head() 

/tmp/ipykernel_6426/2711619272.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  movies_df = movies_df.drop('genres', 1)


,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [5]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Removing timestamp because its not needed

In [6]:
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

/tmp/ipykernel_6426/3809009477.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ratings_df = ratings_df.drop('timestamp', 1)


,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


making a df out of userinputs data of the watched movies

In [7]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


Adding movie Id to input user


In [9]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())] # matching movies
inputMovies = pd.merge(inputId, inputMovies) # merging inputId and input movies
inputMovies = inputMovies.drop('year', 1) # removing year because its not needed
inputMovies

/tmp/ipykernel_6426/406558113.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  inputMovies = inputMovies.drop('year', 1) # removing year because its not needed


,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


Finding users who has seen the same movies (same as inputMovies = the movies that main user has watched)

In [10]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


and group them by id

In [11]:
userSubsetGroup = userSubset.groupby(['userId'])

Lets look at a random numebr for test

In [12]:
userSubsetGroup.get_group(13)

,userId,movieId,rating
479,13,2,2.0
531,13,1274,5.0


Sorting users with more same movies as input movies therefore the top rows are the most relevant 

In [13]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]

[(75,
        userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0),
 (106,
        userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5),
 (686,
         userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0)]

grabbing top 100 users with the same movies to loop through

In [14]:
userSubsetGroup = userSubsetGroup[0:100]

calculating user rating using Pearson Correlation

In [18]:
pearsonCorrelationDict = {} ## tmp storage

for name, group in userSubsetGroup:
    print('name: ',name) # name == userId
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    nRatings = len(group) # amount of ratings
    
    ## get to movies that the person in loop and the input user have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())] 
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()
    
    # calculating pearson alg between 2 users
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    # check that the value is not zero because division by 0 would be infinite 
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


name:  75
name:  106
name:  686
name:  815
name:  1040
name:  1130
name:  1502
name:  1599
name:  1625
name:  1950
name:  2065
name:  2128
name:  2432
name:  2791
name:  2839
name:  2948
name:  3025
name:  3040
name:  3186
name:  3271
name:  3429
name:  3734
name:  4099
name:  4208
name:  4282
name:  4292
name:  4415
name:  4586
name:  4725
name:  4818
name:  5104
name:  5165
name:  5547
name:  6082
name:  6207
name:  6366
name:  6482
name:  6530
name:  7235
name:  7403
name:  7641
name:  7996
name:  8008
name:  8086
name:  8245
name:  8572
name:  8675
name:  9101
name:  9358
name:  9663
name:  9994
name:  10248
name:  10315
name:  10368
name:  10607
name:  10707
name:  10863
name:  11314
name:  11399
name:  11769
name:  11827
name:  12069
name:  12120
name:  12211
name:  12325
name:  12916
name:  12921
name:  13053
name:  13142
name:  13260
name:  13366
name:  13768
name:  13888
name:  13923
name:  13934
name:  14529
name:  14551
name:  14588
name:  14984
name:  15137
name:  15157
nam

In [16]:
pearsonCorrelationDict # dictionary of userIds and ratings (similary ratings to the input user)

{75: 0.8272781516947562,
 106: 0.5860090386731182,
 686: 0.8320502943378437,
 815: 0.5765566601970551,
 1040: 0.9434563530497265,
 1130: 0.2891574659831201,
 1502: 0.8770580193070299,
 1599: 0.4385290096535153,
 1625: 0.716114874039432,
 1950: 0.179028718509858,
 2065: 0.4385290096535153,
 2128: 0.5860090386731196,
 2432: 0.1386750490563073,
 2791: 0.8770580193070299,
 2839: 0.8204126541423674,
 2948: -0.11720180773462392,
 3025: 0.45124262819713973,
 3040: 0.89514359254929,
 3186: 0.6784622064861935,
 3271: 0.26989594817970664,
 3429: 0.0,
 3734: -0.15041420939904673,
 4099: 0.05860090386731196,
 4208: 0.29417420270727607,
 4282: -0.4385290096535115,
 4292: 0.6564386345361464,
 4415: -0.11183835382312353,
 4586: -0.9024852563942795,
 4725: -0.08006407690254357,
 4818: 0.4885967564883424,
 5104: 0.7674257668936507,
 5165: -0.4385290096535153,
 5547: 0.17200522903844556,
 6082: -0.04728779924109591,
 6207: 0.9615384615384616,
 6366: 0.6577935144802716,
 6482: 0.0,
 6530: -0.351605423203

In [19]:
# making df from results
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.827278,75
1,0.586009,106
2,0.832050,686
3,0.576557,815
4,0.943456,1040


find the 50 with highest similarity or perason value to the input user

In [20]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
64,0.961678,12325
34,0.961538,6207
55,0.961538,10707
67,0.960769,13053
4,0.943456,1040


weighted rating from the top users which is calculated by multiplying similarityindex (pearson alg val) and users rating


In [21]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.961678,12325,1,3.5,3.365874
1,0.961678,12325,2,1.5,1.442517
2,0.961678,12325,3,3.0,2.885035
3,0.961678,12325,5,0.5,0.480839
4,0.961678,12325,6,2.5,2.404196


sum of the simlarity index (simlarity rating between user and the input user) and the weighted ratings

In [22]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,38.376281,140.800834
2,38.376281,96.656745
3,10.253981,27.254477
4,0.929294,2.787882
5,11.723262,27.151751


get the mean reconmmendation score by dividing weighted rating by simlarity index

In [23]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.668955,1
2,2.518658,2
3,2.657941,3
4,3.000000,4
5,2.316058,5


and get the top 10 movies that fits best for the input uesr

In [24]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
5073,5.0,5073
3329,5.0,3329
2284,5.0,2284
26801,5.0,26801
6776,5.0,6776
6672,5.0,6672
3759,5.0,3759
3769,5.0,3769
3775,5.0,3775


find the movie location to get movie details by movie id

In [25]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
2200,2284,Bandit Queen,1994
3243,3329,"Year My Voice Broke, The",1987
3669,3759,Fun and Fancy Free,1947
3679,3769,Thunderbolt and Lightfoot,1974
3685,3775,Make Mine Music,1946
4978,5073,"Son's Room, The (Stanza del figlio, La)",2001
6563,6672,War Photographer,2001
6667,6776,Lagaan: Once Upon a Time in India,2001
9064,26801,Dragon Inn (Sun lung moon hak chan),1992
18106,90531,Shame,2011


# Content Based

reading the data again

In [26]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings_sample.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


and remove the year from title and add it to seprate columns again

In [27]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip()) # removing extra spaces arount it
movies_df.head()

/tmp/ipykernel_6426/298990096.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


split the genre list by pipe line

In [28]:
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


and add seprate column for each genre and add corresponding value as binary

In [29]:
moviesWithGenres_df = movies_df.copy()

for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


removing the timestamp from the df again

In [31]:
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

/tmp/ipykernel_6426/3809009477.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ratings_df = ratings_df.drop('timestamp', 1)


,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [32]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


add movie id from movie list to userinput list and dropping genre array and year added from movies data frame

In [ ]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
inputMovies

and add corresponding genre to the userinput movies

In [ ]:
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

cleaning up the data by reseting the index and removing movieId title genre array and year because divided genre columns are the only thing that will be using

In [36]:
userMovies = userMovies.reset_index(drop=True)
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

NameError: name 'userMovies' is not defined

multiplying user rating to the genre matrix to get weigted ratings or user profile

In [35]:
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

NameError: name 'userGenreTable' is not defined

and now we have the user profile we can start matching

In [33]:
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId']) # set movieId as index
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1) # removing the unnecessary information
genreTable.head()

/tmp/ipykernel_6426/4266602852.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1) # removing the unnecessary information


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


multiply the genres by the weights and then take the weighted average

In [34]:
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum()) 
recommendationTable_df.head()

NameError: name 'userProfile' is not defined

In [ ]:
recommendationTable_df = recommendationTable_df.sort_values(ascending=False) # sorting the data 
recommendationTable_df.head()

and find the movielocation to get details based on movieId

In [ ]:
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]